# Amazon Nova Lite Video Understanding

In this notebook we take a quick look at using Amazon Nova for a sample use-case such as analyzing my boxing sparring footage. Replace the S3 path with your video and system prompts with your own, if you would like to test with your own videos.

## Resources/Credits
- Documentation: https://docs.aws.amazon.com/nova/latest/userguide/modalities-video-examples.html (I just directly borrowed most of the code from here)

In [ ]:
import boto3
import json

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client(
    "bedrock-runtime",
    region_name="us-east-1",
)

MODEL_ID = "us.amazon.nova-lite-v1:0"

# Define your system prompt(s).
system_list = [
    {
        "text": "You are an expert boxing analyst. Analyze the sparring footage provided and appropriately answer the query provided by the user."
    }
]

# Define user input
input_prompt = "This is a round of sparring footage with my coach, for context I'm the one in the full sleeves. Based off of the footage tell me my strengths, weaknesses, where I got caught by him specifically and where I need to be better."

#S3 Location: Use for larger files, (Recommended for payload greater than 25MB upto 1GB)
s3_video_path = "Enter S3 payload path here"

# Video format, supported formats atm: MP4, MOV, MKV, WebM, FLV, MPEG, MPG, WMV, 3GP
vid_format = "mov"

# Define a "user" message including both the image and a text prompt.
message_list = [
    {
        "role": "user",
        "content": [
            {
                "video": {
                    "format": vid_format,
                    "source": {
                        "s3Location": {
                            "uri": s3_video_path
                        }
                    }
                }
            },
            {
                "text": input_prompt
            }
        ]
    }
]

# Inference parameters
inf_params = {"maxTokens": 500, "topP": 0.1, "topK": 20, "temperature": 0.3}

# Shape payload
request = {
    "schemaVersion": "messages-v1",
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params,
}

In [ ]:
response = client.invoke_model(modelId=MODEL_ID, body=json.dumps(request))
model_response = json.loads(response["body"].read())

# Parse output text
content_text = model_response["output"]["message"]["content"][0]["text"]
print("\n[Response Content Text]")
print(content_text)